<a href="https://colab.research.google.com/github/Goodbam/Changing-Naver-rising-searches/blob/master/crawling(12_5_11_30AM).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
!pip install lxml
!pip install parse

In [0]:
import requests                 # url request
from lxml.html import parse     # parsing the data as the html form
from io import StringIO         # String I/O module

THRESHOLD = 28 #limit to check whether they are relative or not
NOISE = 9

In [57]:
url = "https://www.naver.com"

# Get html source
text = requests.get(url).text

# parsing the data as the html form
text_source = StringIO(text)    # read the data as string
parsed = parse(text_source)     # source ->parsing the data as the html form 

#find root node
doc = parsed.getroot()

# doc.findall(".//TagName") => find the tag lists
uls = doc.findall('.//ul')

#Get the Rising searches
ul = uls[10]   
spans = ul.findall('.//span')

data_list= []  # to save all the keywords

cnt = 1
for data in spans:
  if cnt % 2 == 0:  
    print(int((cnt+1)/2),'->',data.text_content().strip())  # show the existing rising searches
  cnt +=1  
  data_list.append(data.text_content().strip()) #save each keyword
    


1 -> 백석역
2 -> 이선정
3 -> 서인영 사건
4 -> 수능 성적표
5 -> 이설
6 -> 홍현희
7 -> 진학사
8 -> 양수경
9 -> 이경애 떡볶이
10 -> 2019 수능 만점자
11 -> 정가은
12 -> lj
13 -> 한국교육과정평가원
14 -> 황지만
15 -> 광주형 일자리
16 -> 블레이드앤소울 레볼루션
17 -> 수능
18 -> 미씽 사라진여자
19 -> 한정수
20 -> 수능 등급컷


In [58]:
path='https://search.naver.com/search.naver?sm=top_hty&fbm=1&ie=utf8&query=' #This is the url for surfing

matrix = [] #to count the score of relationship
label = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] #disjoint set

cnt1 = 0

for i in range(0, 20):
  word = data_list[1 + cnt1]
  search = path + word #url searching the keyword
  text = requests.get(search).text # no worry about encoding url
  
  row = []
  cnt2 = 0
  for j in range(0, 20):
    if cnt1 != cnt2: #no need to check when they are equivalent
      target = data_list[1 + cnt2];
      row.append(text.count(target) + 7*word.count(target) - NOISE) 
      
      if row[int(cnt2/2)] >= THRESHOLD: #IF they are relative,
        print(word, ' and ', target,)
        #the representive number should be minimized
        
        #여기 정규화를 해서 row의합으로 각 row의 값을 나누던지 해보자
        #그러려니, 전체적으로 점수 자체가 높은 것은 감안해서 평균을 다시 루트씌우고 곱하든지 고려해보자
        #A와 B 관련 , B C 관련 이면 A C는 관련이 있는건가? 이것도 감안
        
        if label[int(cnt1/2)] < label[int(cnt2/2)]:
          label[int(cnt2/2)] = label[int(cnt1/2)]  
        
        else:
          label[int(cnt1/2)] = label[int(cnt2/2)]
    
    else :
      row.append(0)
    
    cnt2 += 2
  
  matrix.append(row)  
  cnt1 += 2
  

이선정  and  정가은
수능 성적표  and  수능
진학사  and  수능
양수경  and  한정수
2019 수능 만점자  and  수능
정가은  and  이선정
lj  and  이선정
한국교육과정평가원  and  수능
황지만  and  이선정
한정수  and  양수경
수능 등급컷  and  수능


In [60]:

print('\n',label,'\n')
print(matrix)

size = [0]
size = size*20

for i in range(0,20):
  size[i] = sum(matrix[i])
 
print(size)
score =[0]
score = score*20

for i in range(0,20):
  score[label[i] - 1] += 1/(i+1); #label is 1~20 => needed to subtract 1


 [1, 2, 3, 4, 5, 6, 4, 8, 9, 4, 2, 2, 4, 2, 15, 16, 4, 18, 8, 4] 

[[0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 10, 0, 0, 2, 0, 27, 0, 2, 0], [4, 0, 1, 0, 2, 2, 0, 0, 0, 0, 35, 17, 0, 20, 2, 0, 27, 0, 2, 0], [4, 4, 0, 0, 2, 2, 0, 0, 0, 0, 2, 9, 0, 0, 2, 0, 27, 0, 2, 0], [4, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 9, 12, 0, 2, 0, 186, 0, 2, 13], [4, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 13, 0, 0, 2, 0, 27, 0, 2, 0], [4, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 7, 0, 0, 2, 0, 27, 0, 2, 0], [4, 0, 0, 7, 2, 2, 0, 0, 0, 0, 0, 10, 5, 0, 2, 0, 105, 0, 2, 16], [4, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 7, 0, 0, 2, 0, 27, 0, 45, 0], [4, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 7, 0, 0, 2, 0, 27, 0, 2, 0], [4, 0, 0, 0, 2, 2, 1, 0, 0, 0, 0, 7, 1, 0, 2, 0, 187, 0, 2, 11], [4, 29, 1, 0, 2, 2, 0, 0, 0, 0, 0, 9, 0, 0, 2, 0, 27, 0, 2, 0], [4, 66, 0, 0, 2, 2, 0, 0, 0, 0, 12, 0, 0, 16, 2, 0, 27, 0, 2, 0], [4, 0, 0, 5, 2, 2, 0, 0, 0, 0, 0, 7, 0, 0, 2, 0, 74, 0, 2, 5], [4, 90, 0, 0, 2, 2, 0, 0, 0, 0, 3, 17, 0, 0, 2, 0, 27, 0, 2, 0], [4, 0, 0, 0, 2, 2, 0, 0, 0, 0

In [61]:

#RESULT
print('\n☆급상승 검색어 순위☆\n')
cnt = 1;

#sorting according to SCORE
for i in range(0,20):
  max = 0
  for j in range(1,20):
    if score[j] > score[max]:
      max = j;
  
  relative_word = ''
  
  for j in range(0,20):
    if label[j] == max + 1:
      relative_word += data_list[1 + 2*j] + ' | ' 
  
  score[max] = 0;
  
  print(cnt, '. ', relative_word[0:-3])
  cnt += 1
  if(score.count(0) == 20):
    break  


☆급상승 검색어 순위☆

1 .  백석역
2 .  이선정 | 정가은 | lj | 황지만
3 .  수능 성적표 | 진학사 | 2019 수능 만점자 | 한국교육과정평가원 | 수능 | 수능 등급컷
4 .  서인영 사건
5 .  이설
6 .  양수경 | 한정수
7 .  홍현희
8 .  이경애 떡볶이
9 .  광주형 일자리
10 .  블레이드앤소울 레볼루션
11 .  미씽 사라진여자
